# Ejercicio 03: Cálculo de Relevancia y Ranking de Documentos

El objetivo de este ejercicio es calcular analíticamente la relevancia de cada documento en un corpus y luego ordenar (rankear) los documentos basándonos en esa puntuación de relevancia para tres consultas específicas. Seguirás los siguientes pasos:

Descripción del Ejercicio

1. Procesamiento del Corpus:
    * Leer y parsear el archivo XML proporcionado que contiene el corpus de documentos con sus metadatos (keywords, autor y fecha).

2. Procesamiento de las Consultas:
    * Definir las consultas proporcionadas.
    * Extraer las palabras clave de cada consulta.

3. Cálculo de Relevancia:
    * Utilizar métricas de similitud (Similitud Coseno y Jaccard) entre la representación vectorial de los documentos y las de las consultas.
    * Calcular la puntuación de relevancia para cada documento del corpus respecto a cada consulta.

4. Ranking de Documentos:
    * Ordenar los documentos en función de su puntuación de relevancia de mayor a menor.
    * Mostrar el ranking de documentos para cada consulta.

In [4]:
# Importar las librerías necesarias
import xml.etree.ElementTree as ET

# Definir las consultas
queries = {
    1: "Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios",
    2: "Actividades extracurriculares y bienestar emocional en el campus universitario",
    3: "Estrategias universitarias para reducir el estrés en estudiantes"
}

# Función para procesar el texto y extraer palabras clave
def process_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Reemplazar caracteres no alfanuméricos por espacios
    import re
    text = re.sub(r'[^a-záéíóúñü]+', ' ', text)
    # Tokenizar y eliminar palabras vacías si es necesario
    tokens = text.strip().split()
    return set(tokens)

# Paso 1: Leer y parsear el archivo XML
def parse_corpus(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    corpus = {}
    for doc in root.findall('document'):
        doc_id = int(doc.get('id'))
        title = doc.find('title').text
        keywords = doc.find('keywords').text
        author = doc.find('author').text
        date = doc.find('date').text
        # Procesar las palabras clave
        keyword_set = process_text(keywords)
        corpus[doc_id] = {
            'title': title,
            'keywords': keyword_set,
            'author': author,
            'date': date
        }
    return corpus

In [6]:
corpus = parse_corpus('../content/sample_data/03ranking_corpus.xml')
corpus

{1: {'title': 'El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.',
  'keywords': {'crónica',
   'médica',
   'salud',
   'tecnología',
   'telemedicina',
   'tratamiento'},
  'author': 'Dr. Juan Pérez',
  'date': '2023-01-15'},
 2: {'title': 'Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.',
  'keywords': {'académico',
   'estudiantes',
   'mental',
   'nutrición',
   'rendimiento',
   'salud'},
  'author': 'Dra. María López',
  'date': '2023-02-10'},
 3: {'title': 'Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus.',
  'keywords': {'amistad',
   'bienestar',
   'campus',
   'estudiantil',
   'relaciones',
   'sociales'},
  'author': 'Miguel Rodríguez',
  'date': '2023-03-05'},
 4: {'title': 'El rol de las bibliotecas universitarias en el fomento de la investigación académica.',
  'keywords': {'academia',
   'bibliotecas',
   'investigación',
   'recursos

## Caclculo de relevancia

In [7]:
query_keywords = {qid: process_text(query) for qid, query in queries.items()}

In [8]:
document_texts = [" ".join(doc['keywords']) for doc in corpus.values()]

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
# Vectorizar documentos y consultas usando TF-IDF
vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform(document_texts)
query_vectors = vectorizer.transform([" ".join(query) for query in query_keywords.values()])


In [10]:
# Función para calcular similitud de Jaccard
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union) if union else 0

In [11]:
# Calcular relevancia y similitud entre cada consulta y documento
relevance_scores = {}
for qid, query_vector in enumerate(query_vectors):
    cos_sim = cosine_similarity(doc_vectors, query_vector).flatten()
    jaccard_sim = [jaccard_similarity(corpus[doc_id]['keywords'], query_keywords[qid + 1]) for doc_id in corpus]
    relevance_scores[qid + 1] = {
        doc_id: (cos_sim[idx], jaccard_sim[idx])
        for idx, doc_id in enumerate(corpus)
    }


## Ranking de Documentos


In [12]:
# Ordenar documentos según la relevancia para cada consulta
ranked_results = {}
for qid, scores in relevance_scores.items():
    ranked_results[qid] = sorted(scores.items(), key=lambda x: (x[1][0], x[1][1]), reverse=True)

# Mostrar el ranking de documentos para cada consulta
for qid, ranking in ranked_results.items():
    print(f"\nRanking de documentos para la consulta {qid}: '{queries[qid]}'")
    for doc_id, (cos_score, jacc_score) in ranking:
        doc_info = corpus[doc_id]
        print(f"  Documento ID {doc_id} - Título: {doc_info['title']}")
        print(f"    Similitud Coseno: {cos_score:.4f}")
        print(f"    Similitud Jaccard: {jacc_score:.4f}")


Ranking de documentos para la consulta 1: 'Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios'
  Documento ID 7 - Título: La importancia del sueño en la salud mental y el rendimiento académico en jóvenes universitarios.
    Similitud Coseno: 0.6145
    Similitud Jaccard: 0.3846
  Documento ID 2 - Título: Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.
    Similitud Coseno: 0.5484
    Similitud Jaccard: 0.3846
  Documento ID 14 - Título: Cómo el acceso a servicios de salud mental en la universidad puede mejorar el desempeño académico.
    Similitud Coseno: 0.4610
    Similitud Jaccard: 0.3571
  Documento ID 29 - Título: Análisis de los hábitos de sueño y su relación con el rendimiento en exámenes universitarios.
    Similitud Coseno: 0.4082
    Similitud Jaccard: 0.2500
  Documento ID 13 - Título: Estrategias para mejorar la salud mental en estudiantes universitarios: el rol del apoyo psicológico y los